# Testing mimic_III_api routes

**App Source Code:** https://github.com/LiamWoodRoberts/mimic_III_api

**Hosted:** https://medication-ner-app.herokuapp.com/

In [1]:
# Packages
import pandas as pd
import re
import spacy
from spacy import displacy
import requests
import numpy as np
import json

In [2]:
text = "DISCHARGE MEDICATIONS: \n1. Aspirin 500 mg tablet twice (2) a day as needed\n2. Oxacillin 2 mg\n"
url = "https://medication-ner-app.herokuapp.com/"
r = requests.post(f"{url}api/get_tags",data = {"text":text}).json()
r

[[24, 26, 'ORDINAL', '1.'],
 [27, 34, 'DRUG', 'aspirin'],
 [39, 41, 'UNIT', 'mg'],
 [56, 58, 'ORDINAL', '2.'],
 [59, 64, 'Frequency', 'a day'],
 [65, 74, 'Frequency', 'as needed'],
 [75, 77, 'ORDINAL', '2.'],
 [78, 87, 'DRUG', 'oxacillin'],
 [88, 89, 'CARDINAL', '2']]

In [3]:
def show_ents(text,entities):
    ents = [{"start":x[0],"end":x[1],"label":x[2]} for x in entities]
    ex = [{"text":text,
       "ents":ents}
         ]
    colors = {"DRUG": "rgb(60,180,240)","DOSE":"rgb(240,180,60)","ROUTE":"rgb(200,200,200)"}
    options = {"colors":colors}
    html = displacy.render(ex,style="ent",manual=True,options=options,jupyter=True)
    return

In [4]:
show_ents(text,r)

### Available Routes:

get_tags - Accepts a block of text and returns entity tags and locations within the text.

drugs_and_conditions - Accepts a block of text and returns a list of all known drugs and conditions within the text.

In [5]:
def api_call(text=None,local=True,route="get_tags"):
    if text == None:
        text = input()
        
    if local:
        url = "http://localhost:5000/"
    else:
        url = "https://medication-ner-app.herokuapp.com/"
    
    r = requests.post(f"{url}api/{route}",data = {"text":text}).json()
    if route=="get_tags":
        show_ents(text,r)
    return r

In [6]:
api_call(text,route="get_tags")

[[24, 26, 'ORDINAL', '1.'],
 [27, 34, 'DRUG', 'aspirin'],
 [39, 41, 'UNIT', 'mg'],
 [56, 58, 'ORDINAL', '2.'],
 [59, 64, 'Frequency', 'a day'],
 [65, 74, 'Frequency', 'as needed'],
 [75, 77, 'ORDINAL', '2.'],
 [78, 87, 'DRUG', 'oxacillin'],
 [88, 89, 'CARDINAL', '2']]

In [7]:
api_call(text,route="drugs_and_conditions")

{'DRUGS': ['aspirin', 'oxacillin'], 'CONDITIONS': []}

---